### 近隣1kmに入る物件の特徴量から、新規特徴量を生成する！

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold
import xgboost
from catboost import CatBoostRegressor
from catboost import Pool

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
building = pd.read_csv('../data/handmade_data/building.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']
over110m2 = pd.read_csv('../data/handmade_data/over110m2.csv')
over100m2 = pd.read_csv('../data/handmade_data/over100m2.csv')
over90m2 = pd.read_csv('../data/handmade_data/over90m2.csv')
bt75and100m2 = pd.read_csv('../data/handmade_data/bt75and100m2.csv')
bt50and75m2 = pd.read_csv('../data/handmade_data/bt50and75m2.csv')
sepa_bath_toilets = pd.read_csv('../data/handmade_data/sepa_bath_toilets.csv')
sepa_bath_toilets.columns = ['バス・トイレ別']
washlets = pd.read_csv('../data/handmade_data/washlets.csv')
area_per_rooms = pd.read_csv('../data/handmade_data/area_per_rooms.csv')
kitchen_points = pd.read_csv('../data/handmade_data/kitchen_points.csv')
equipments = pd.read_csv('../data/handmade_data/equipments.csv')
latitude_longitude = pd.read_csv('../data/handmade_data/latitude_longitude.csv')
broadcasts = pd.read_csv('../data/handmade_data/broadcasts.csv')
stations_te = pd.read_csv('../data/handmade_data/stations_te.csv')

rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_contract_span2.columns = ['契約期間']
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_building = pd.read_csv('../data/test_data/test_building.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']
test_over110m2 = pd.read_csv('../data/test_data/test_over110m2.csv')
test_over100m2 = pd.read_csv('../data/test_data/test_over100m2.csv')
test_over90m2 = pd.read_csv('../data/test_data/test_over90m2.csv')
test_bt75and100m2 = pd.read_csv('../data/test_data/test_bt75and100m2.csv')
test_bt50and75m2 = pd.read_csv('../data/test_data/test_bt50and75m2.csv')
test_sepa_bath_toilets = pd.read_csv('../data/test_data/test_sepa_bath_toilets.csv')
test_sepa_bath_toilets.columns = ['バス・トイレ別']
test_washlets = pd.read_csv('../data/test_data/test_washlets.csv')
test_area_per_rooms = pd.read_csv('../data/test_data/test_area_per_rooms.csv')
test_kitchen_points = pd.read_csv('../data/test_data/test_kitchen_points.csv')
test_equipments = pd.read_csv('../data/test_data/test_equipments.csv')
test_latitude_longitude = pd.read_csv('../data/test_data/test_latitude_longitude.csv')
test_broadcasts = pd.read_csv('../data/test_data/test_broadcasts.csv')
test_stations_te = pd.read_csv('../data/test_data/test_stations_te.csv')

Duplicate key in file PosixPath('/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : IPAexGothic')


In [3]:
stations.index[13]

13

In [4]:
list = [[1,2],[3,4],[5,6]]

In [5]:
df = pd.DataFrame(list)

In [6]:
df[1].sort_values(ascending=False)[:2]

2    6
1    4
Name: 1, dtype: int64

In [80]:
# 近い物件を10件取得する関数
from sklearn import neighbors


def get_neibours(df, n):
    this_lat = df.loc[n]['平均緯度']
    this_lon = df.loc[n]['平均経度']

    distances = []
    for i in range(len(df)):
        if i != n:
            distance_square = (this_lat*1000 - df.loc[i]['平均緯度']*1000)**2 + (this_lon*1000 - df.loc[i]['平均経度']*1000)**2
            print(distance_square)
            distances.append([df.index[i],distance_square])#比較した物件のidと、そこまでの距離を渡す

    # 近い順に並び替えて、近いほうから10件のidを取得する
    distances = pd.DataFrame(distances)
    neigbours = distances[1].sort_values()[:10]
    neighbour_ids = neigbours.index
    return neighbour_ids

# 近い物件の平均面積
def get_mean_area(df, neighbour_ids):
    return np.mean(df.loc[neighbour_ids]['面積'])

In [81]:
df_train = pd.read_csv('../data/outer_data/train.csv')

In [82]:
df = pd.concat([df_train,area_size, latitude_longitude[['平均緯度','平均経度']]],axis=1)

In [83]:
get_neibours(df,0)

10032.28188617879
9008.716357907115
7874.364582957937
20572.113704013274
4879.740295071678
19951.790435166688
15958.949184152993
1508.1342422088292
176.0875663474433
25762.30963092255
25844.567688747506
6752.878073820792
10303.904839253022
8423.635203430522
nan
2887.453562523432
28853.242314969233
3466.5890915196433
7862.47719083207
4238.138556623415
3281.460676440579
9758.449176924943
30132.5733263941
25844.567688747506
1713.4842035669583
4421.98514167188
9688.25603697524
22984.51627555638
8703.6580309184
1293.9655773354473
26761.90469549826
18459.117322834358
24729.63407076341
4018.504670356576
14574.153082222416
19973.001773076125
4879.740295071678
2249.3008027969613
18233.357330855724
1517.3319260887517
7146.679633105768
29224.61159214838
29224.61159214838
24063.718601792
5195.467148887196
9763.379350604497
8265.178583482388
4041.1280967804673
4550.998238428432
24563.38750370332
2850.467024270781
5576.439438776612
18957.748784254312
21934.717389316436
12581.503767151591
7745.053331

Int64Index([14576, 8086, 19037, 30140, 10959, 750, 29387, 10954, 24880, 20982], dtype='int64')

In [76]:
df.loc[0]

id                                                        1
賃料                                                    75000
所在地                                             東京都北区滝野川３丁目
アクセス      都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...
間取り                                                      1K
築年数                                                   9年9ヶ月
方角                                                       南東
面積                                                  20.01m2
所在階                                                 1階／12階建
バス・トイレ     専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座
キッチン                        ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
放送・通信                    インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ
室内設備      エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...
駐車場                                                 駐輪場\t空有
周辺環境      【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...
建物構造                                           RC（鉄筋コンクリート）
契約期間                                    

In [85]:
df.loc[20981]

id                                                    20982
賃料                                                    96000
所在地                                       東京都渋谷区代々木４丁目62-18
アクセス      京王新線\t初台駅\t徒歩6分\t\t小田急小田原線\t参宮橋駅\t徒歩8分\t\t小田急小...
間取り                                                      1K
築年数                                                   8年5ヶ月
方角                                                       南東
面積                                                  20.02m2
所在階                                           3階／13階建（地下1階）
バス・トイレ    専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...
キッチン               ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯／\t独立キッチン
放送・通信                       光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ
室内設備      冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気...
駐車場                                     駐輪場\t空有\tバイク置き場\t空有
周辺環境      【コンビニ】 523m\t【コンビニ】 479m\t【コンビニ】 468m\t【スーパー】 ...
建物構造                                           RC（鉄筋コンクリート）
契約期間                                    

In [79]:
df.loc[8085]

id                                                     8086
賃料                                                   106000
所在地                                        東京都大田区南千束３丁目4-12
アクセス      東急目黒線\t大岡山駅\t徒歩5分\t\t東急大井町線\t北千束駅\t徒歩10分\t\t東急...
間取り                                                      1R
築年数                                                   4年3ヶ月
方角                                                        北
面積                                                  25.23m2
所在階                                                  3階／4階建
バス・トイレ    専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立...
キッチン                        ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
放送・通信                              インターネット対応／\tインターネット使用料無料
室内設備      エアコン付\tシューズボックス／\tフローリング／\t24時間換気システム\t／\t室内洗濯...
駐車場                              駐輪場\t空有\t駐車場\t無\tバイク置き場\t無
周辺環境      【スーパー】 388m\t【スーパー】 422m\t【コンビニ】 50m\t【コンビニ】 255m
建物構造                                           RC（鉄筋コンクリート）
契約期間                                    

In [67]:
df.loc[19629]

id                                                    19630
賃料                                                    97000
所在地                                               東京都葛飾区東水元
アクセス      常磐線\t金町(東京都)駅\t/バス(11分)水元そよかぜ園下車徒歩6分\t\t常磐線\t金...
間取り                                                    2LDK
築年数                                                      新築
方角                                                      NaN
面積                                                  55.34m2
所在階                                                  1階／3階建
バス・トイレ                   バス・トイレ別／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立
キッチン                                 システムキッチン\t／\tカウンターキッチン
放送・通信                                             インターネット対応
室内設備       エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場\t都市ガス
駐車場                                                  駐車場\t無
周辺環境                                                    NaN
建物構造                                                   軽量鉄骨
契約期間                                    